## 02 Artifact_Removal
NOTE: this notebook does not include psd and epoching was done before the removal of irrelevant frequencies.

drift: 0.01-1 Hz <br>
no_drift: 1-40 Hz

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
from mne import Epochs, find_events
from IPython.display import clear_output

### 1. Loading Data

#### 1.1 Load pd

In [9]:
par = "par1"
file = "1-Chaichan_1_2021-04-07-06.24.14"
drift = "no_drift"

Here I decided to keep stuff in a form of dictionary where **keys** indicate the **task and time**

In [10]:
path_task_types= ["_visual", "_imagery"]
# Just wanted to make the names look nice

task_types = ["visual", "imagery"]

dfs_task_types = {}

for i,path_task_type in enumerate(path_task_types):
    path = "../data/pd/round2/{par}/{file}{path_task_type}.pkl".format(par = par, file = file, path_task_type = path_task_type)
    f = open(path,'rb')
    df = pickle.load(f)
    print(df['Marker'].unique())
    dfs_task_types["{task_types}".format(task_types = task_types[i])] = df

[0 1 2 3]
[0 1 2 3]


### 2. Artifact Removal

To be installed!!!!!!!!! <br>
pip3 install numpy==1.18 <br> 
pip3 install scipy==1.4.1 <br>
pip3 install scikit-learn==0.21.3 <br>

Artifacts that are restricted to a narrow frequency range can sometimes be repaired by filtering the data. Two examples of frequency-restricted artifacts are slow drifts and power line noise. Here we illustrate how each of these can be repaired by filtering.

But first we gonna use Python MNE as it provides many useful methods for achieving these tasks.  So first, we gonna transform our pandas to mne type.  Here is the function transforming df to raw mne.

In [ ]:
import mne
from mne import create_info
from mne.io import RawArray

def df_to_raw(df):
    sfreq = 125
    ch_names = list(df.columns)
    ch_types = ['eeg'] * (len(df.columns) - 1) + ['stim']
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    df = df.T  #mne looks at the tranpose() format
    df[:-1] *= 1e-6  #convert from uVolts to Volts (mne assumes Volts data)

    info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)

    raw = mne.io.RawArray(df, info)
    raw.set_montage(ten_twenty_montage)

    #try plotting the raw data of its power spectral density
    raw.plot_psd()

    return raw

Transform df to raw mne.

In [ ]:
raws = {}
for task_type in task_types:
    print(f"========================= {task_type} ==========================")
    raws[task_type] = df_to_raw(dfs_task_types[task_type])

#### 1. Power line noise

Some artifacts are restricted to certain frequencies and can therefore be fixed by filtering. An artifact that typically affects only some frequencies is due to the power line.

Power-line noise is a noise created by the electrical network. It is composed of sharp peaks at 50Hz (or 60Hz depending on your geographical location). Some peaks may also be present at the harmonic frequencies, i.e. the integer multiples of the power-line frequency, e.g. 100Hz, 150Hz, … (or 120Hz, 180Hz, …).

Remove the 50Hz power line noise in Thailand.  We will also be remove its harmonics, i.e., 100Hz, 150Hz, etc.  Since our signal is 62.5Hz (125Hz / 2 according to Nyquist Theorem), we don't need to run the harmonics but simply notch the 50Hz signal.

In [ ]:
for task_type in task_types:
    print(f"========================= {task_type} ==========================")    
    raws[task_type].notch_filter(50, filter_length='auto', phase='zero') #250/2 based on Nyquist Theorem

In [ ]:
# #observe that the 50Hz noise is now gone, yay!
# for task_type in task_types:
#     print(f"========================= {task_type} ==========================")
#     raws[task_type].plot_psd()

#### 2. Slow drift

Low-frequency drifts in raw data can usually be spotted by plotting a fairly long span of data with the plot() method, though it is helpful to disable channel-wise DC shift correction to make slow drifts more readily visible. Here we plot 3600 seconds (since we perform a 60 minutes experiment), showing all the eeg channels:

In [ ]:
for task_type in task_types:
    print(f"========================= {task_type} ==========================")
    eeg_channels = mne.pick_types(raws[task_type].info, eeg=True)
    raws[task_type].plot(duration=5400, order=eeg_channels)

Notice that there are a lot of vertical black lines.  Those are drifts.  We can usually remove using low frequency high pass filter.  Here let's try 0.1, 0.2 and 1 Hz.

**change duration!!!!!!!!!!!**

In [ ]:
duration = 5400
for cutoff in (0.1, 0.2, 1):
    for task_type in task_types:
        print(f"========================= {task_type} ==========================")
        raw_highpass = raws[task_type].copy().filter(l_freq=cutoff, h_freq=None)
        fig = raw_highpass.plot(duration=duration, order=eeg_channels)
        fig.subplots_adjust(top=0.9)
        fig.suptitle('High-pass filtered at {} Hz'.format(cutoff), size='xx-large',
                     weight='bold')

Looks like 1Hz was quite quite good to fully remove the slow drifts. Usually, 1Hz is a good measure since most of the brain frequency lies around 1 to 40Hz.  Given that, we shall filter our brain signal to 1 to 40Hz.

In [ ]:
for task_type in task_types:
    print(f"========================= {task_type} ==========================")
    if drift == "drift":
        raws[task_type].filter(0.01, 1, method='iir')
    else:
        raws[task_type].filter(1, 40, method='iir')

#     raws[task_type].plot_psd()

#### 3. Independent component analysis

Independent components analysis (ICA) is a technique for estimating independent source signals from a set of recordings in which the source signals were mixed together in unknown ratios. A common example of this is the problem of blind source separation: with 3 musical instruments playing in the same room, and 3 microphones recording the performance (each picking up all 3 instruments, but at varying levels), can you somehow “unmix” the signals recorded by the 3 microphones so that you end up with a separate “recording” isolating the sound of each instrument?

It is not hard to see how this analogy applies to EEG/MEG analysis: there are many “microphones” (sensor channels) simultaneously recording many “instruments” (blinks, heartbeats, activity in different areas of the brain, muscular activity from jaw clenching or swallowing, etc). As long as these various source signals are statistically independent and non-gaussian, it is usually possible to separate the sources using ICA, and then re-construct the sensor signals after excluding the sources that are unwanted.

MNE-Python implements three different ICA algorithms: fastica (the default), picard, and infomax. FastICA and Infomax are both in fairly widespread use; Picard is a newer (2017) algorithm that is expected to converge faster than FastICA and Infomax, and is more robust than other algorithms in cases where the sources are not completely independent, which typically happens with real EEG/MEG data

The ICA interface in MNE-Python is similar to the interface in scikit-learn: some general parameters are specified when creating an ICA object, then the ICA object is fit to the data using its fit() method. The results of the fitting are added to the ICA object as attributes that end in an underscore (_), such as ica.mixing_matrix_ and ica.unmixing_matrix_. After fitting, the ICA component(s) that you want to remove must be chosen, and the ICA fit must then be applied to the Raw or Epochs object using the ICA object’s apply() method.

After visualizing the Independent Components (ICs) and excluding any that capture artifacts you want to repair, the sensor signal can be reconstructed using the ICA object’s apply() method. By default, signal reconstruction uses all of the ICs (less any ICs listed in ICA.exclude) plus all of the PCs that were not included in the ICA decomposition (i.e., the “PCA residual”). If you want to reduce the number of components used at the reconstruction stage, it is controlled by the n_pca_components parameter (which will in turn reduce the rank of your data; by default n_pca_components = max_pca_components resulting in no additional dimensionality reduction). 

Before we run the ICA, an important step is filtering the data to remove low-frequency drifts, which can negatively affect the quality of the ICA fit. The slow drifts are problematic because they reduce the independence of the assumed-to-be-independent sources (e.g., during a slow upward drift, the neural, heartbeat, blink, and other muscular sources will all tend to have higher values), making it harder for the algorithm to find an accurate solution. **A high-pass filter with 1 Hz cutoff frequency is recommended before performing ICA.  However when we transform ICA (signal reconstruction), we reconstruct based on the original signal that did not perform 1Hz cutoff.** However, because filtering is a linear operation, the ICA solution found from the filtered signal can be applied to the unfiltered signal, so we’ll keep a copy of the unfiltered Raw object around so we can apply the ICA solution to it later.

In [ ]:
# #filtering to remove slow drifts; also make copy of raw for later signal reconstruction
# from mne.preprocessing import ICA

# filt_raw = raw.copy()
# filt_raw.load_data().filter(l_freq=1., h_freq=None)
# filt_raw.plot_psd()

Now we’re ready to set up and fit the ICA. **We’ll run ICA with n_components=16 since we only have 16 channels.**  If we have more, then we may pick a smaller number.  How small?  There is no answer, but clearly, we want to try until we can see the eye, muscle components, etc.  But too large the component will slow the fit() process.

ICA fitting is not deterministic (e.g., the components may get a sign flip on different runs, or may not always be returned in the same order), so we’ll also specify a random seed so that we get identical results each time this file is run.

In [ ]:
# # set up and fit the ICA
# ica = ICA(n_components=16, random_state=32)
# ica.fit(filt_raw)

Now we can examine the ICs to see what they captured. plot_sources() will show the time series of the ICs. Note that in our call to plot_sources() we can use the original, unfiltered Raw object:

In [ ]:
# ica.plot_sources(filt_raw)
# ica.plot_components()

From the picture above, component 0 and 1 are suspicious of eye artifact. The time series shows that the first component is quite stable which is not indicative of brain components.  The time series also shows that the second component feels like it is a eye component.   For details how to look at this, please refer to https://labeling.ucsd.edu/tutorial/labels.   We shall plot the properties of ICA component 0 and 1 more closely.

In [ ]:
# ica.plot_properties(filt_raw, picks=[0, 1])

Clearly, looking at the time series data, the first component is definitely not a brain component.  It could be due to the fault of the device or the setup.  For the second component, it is not straightforward as typically eye artifacts are mostly spikes, but here it looks like a mix of brain and eye.  

For more in-depth analysis, we can alo plot an overlay of the original signal against the reconstructed signal with the artifactual ICs excluded, using plot_overlay():

In [ ]:
#test excluding 0, 1 component
# ica.plot_overlay(filt_raw, exclude=[0, 1], picks='eeg')

Seems like removing component 0 does not even change the signal, meaning that component 0 can be safely removed.  Removing component 1 also seems to help removing the big spike thus is recommended.

Once we’re certain which components we want to exclude, we can specify that manually by setting the **ica.exclude** attribute. Similar to marking bad channels, merely setting ica.exclude doesn’t do anything immediately (it just adds the excluded ICs to a list that will get used later when it’s needed). Once the exclusions have been set, ICA methods like plot_overlay() will exclude those component(s) even if no exclude parameter is passed, and the list of excluded components will be preserved when using mne.preprocessing.ICA.save() and mne.preprocessing.read_ica().

In [ ]:
# ica.exclude = [0, 1] #we want to cut down the 0, 1 component, then apply(self) to reconstruct the signal

Once we’re confident about which component(s) we want to remove, we pass them as the exclude parameter and then apply the ICA to the raw signal. The apply() method requires the raw data to be loaded into memory (by default it’s only read from disk as-needed), so we’ll use load_data() first. We’ll also make a copy of the Raw object so we can compare the signal before and after artifact removal side-by-side:

In [ ]:
# ica.apply() changes the Raw object in-place, so let's make a copy first for comparison:
# orig_raw = raw.copy()  #we apply ica to raw
# ica.apply(raw)

Let's plot the original raw against the filtered raw

In [ ]:
# orig_raw.plot(order=eeg_channels, duration=20)
# raw.plot(order=eeg_channels, duration=20)

### 2. Epoching and Save Data as np

In [ ]:
def getEpochs(raw, event_id, tmin, tmax, picks):

    #epoching
    events = find_events(raw)
    
    #reject_criteria = dict(mag=4000e-15,     # 4000 fT
    #                       grad=4000e-13,    # 4000 fT/cm
    #                       eeg=100e-6,       # 150 μV
    #                       eog=250e-6)       # 250 μV

    reject_criteria = dict(eeg=100e-6)  #most voltage in this range is not brain components

    epochs = Epochs(raw, events=events, event_id=event_id, 
                    tmin=tmin, tmax=tmax, baseline=None, preload=True,verbose=False, picks=picks)  #8 channels
    print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)

    return epochs

In [ ]:
#this one requires expertise to specify the right tmin, tmax

event_id = {'0': 1, '1' : 2, '2': 3}
tmin = 0.03 #0
tmax = 1.5
epochs = {}
for task_type in task_types:
    picks= eeg_channels
    epochs = getEpochs(raws[task_type], event_id, tmin, tmax, picks)
    X = epochs.get_data()
    y = epochs.events[:, -1]
    y = y-1
#     np.save("../data/np/round2/{par}/{drift}/{file}_{task_type}_X".format(par=par, drift = drift, file=file, task_type = task_type), X)
#     np.save("../data/np/round2/{par}/{drift}/{file}_{task_type}_y".format(par=par, drift = drift, file=file, task_type = task_type), y)